### LIGHT GMB MODEL

In [178]:
import warnings
warnings.filterwarnings('ignore')

In [179]:
import lightgbm as lgb
from datetime import datetime
from typing import Tuple
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import optuna
from sklearn.model_selection import KFold,TimeSeriesSplit
from sklearn.pipeline import make_pipeline, Pipeline
from datetime import timedelta
from typing import List

In [180]:
df = pd.read_parquet('/Users/chetanhalai/Documents/code base for projects/4) uber/FINAL TAXI PROJECT 2023/tabular_data_final.parquet')
df.head()

,rides_previous_648_hour,rides_previous_647_hour,rides_previous_646_hour,rides_previous_645_hour,rides_previous_644_hour,rides_previous_643_hour,rides_previous_642_hour,rides_previous_641_hour,rides_previous_640_hour,rides_previous_639_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,19.0,28.0,43.0,33.0,12.0,3.0,2.0,1.0,1.0,1.0,...,3.0,4.0,5.0,3.0,8.0,18.0,22.0,2023-01-28,4,58.0
1,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,0.0,...,5.0,5.0,7.0,5.0,8.0,15.0,26.0,2023-01-29,4,53.0
2,2.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,2.0,3.0,...,3.0,1.0,5.0,2.0,1.0,2.0,0.0,2023-01-30,4,2.0
3,3.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,6.0,2.0,...,2.0,1.0,1.0,1.0,0.0,0.0,2.0,2023-01-31,4,0.0
4,3.0,16.0,28.0,21.0,12.0,5.0,4.0,2.0,2.0,3.0,...,4.0,2.0,3.0,0.0,5.0,1.0,2.0,2023-01-28,7,1.0


In [181]:
def train_test_split(
    df: pd.DataFrame,
    cutoff_date: datetime,
    target_column_name: str,
    ) -> Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]:
    """
    """
    train_data = df[df.pickup_hour < cutoff_date].reset_index(drop=True)
    test_data = df[df.pickup_hour >= cutoff_date].reset_index(drop=True)

    X_train = train_data.drop(columns=[target_column_name])
    y_train = train_data[target_column_name]
    X_test = test_data.drop(columns=[target_column_name])
    y_test = test_data[target_column_name]

    return X_train, y_train, X_test, y_test

In [182]:
X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2023, 4, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(2330, 650)
y_train.shape=(2330,)
X_test.shape=(2592, 650)
y_test.shape=(2592,)


In [183]:
# use only past rides data
past_rides_columns = [c for c in X_train.columns if c.startswith('rides_')]
X_train_only_numeric = X_train[past_rides_columns]

In [184]:
#convert to numpy values
X_train_only_numeric = X_train_only_numeric.values

In [185]:
#convert to numpy values
y_train = y_train.values

In [186]:
model = lgb.LGBMRegressor()
model.fit(X_train_only_numeric, y_train)

LGBMRegressor()

In [187]:
X_test_only_numeric = X_test[past_rides_columns]
predictions = model.predict(X_test_only_numeric)

test_mae = mean_absolute_error(y_test, predictions)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=3.8993


### Feature Engineering

As date time variables cant be ingested by bosting algorithms i will do a little bit of feature enginnering - and create alternative columns for our original 'Pick-up hour' column - create latitude and longitude for location id's

In [188]:
#Average number of ride for the previous 3 weeks
def average_rides_last_3_weeks(X: pd.DataFrame) -> pd.DataFrame:
    """
    Adds one column with the average rides from
    - 7 days ago
    - 14 days ago
    - 21 days ago
    """
    X['average_rides_last_3_weeks'] = 0.33*(
        X[f'rides_previous_{7*24}_hour'] + \
        X[f'rides_previous_{2*7*24}_hour'] + \
        X[f'rides_previous_{3*7*24}_hour']
    )

    return X

In [189]:
from sklearn.preprocessing import FunctionTransformer

In order t create a pipline - Pipeline is to execute the repeated functions sequentially - pipleine does not take customer functions directly - It will only accept custome functions through 'Function tranformer method

that is why we are adding our function to the transformer]

- Fit_transform -

In [190]:
add_feature_average_rides_last_3_weeks = FunctionTransformer(
    average_rides_last_3_weeks, validate=False)

add_feature_average_rides_last_3_weeks.fit_transform(X_train)

,rides_previous_648_hour,rides_previous_647_hour,rides_previous_646_hour,rides_previous_645_hour,rides_previous_644_hour,rides_previous_643_hour,rides_previous_642_hour,rides_previous_641_hour,rides_previous_640_hour,rides_previous_639_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,average_rides_last_3_weeks
0,19.0,28.0,43.0,33.0,12.0,3.0,2.0,1.0,1.0,1.0,...,3.0,4.0,5.0,3.0,8.0,18.0,22.0,2023-01-28,4,40.920002
1,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,0.0,...,5.0,5.0,7.0,5.0,8.0,15.0,26.0,2023-01-29,4,47.520000
2,2.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,2.0,3.0,...,3.0,1.0,5.0,2.0,1.0,2.0,0.0,2023-01-30,4,2.970000
3,3.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,6.0,2.0,...,2.0,1.0,1.0,1.0,0.0,0.0,2.0,2023-01-31,4,1.320000
4,3.0,16.0,28.0,21.0,12.0,5.0,4.0,2.0,2.0,3.0,...,4.0,2.0,3.0,0.0,5.0,1.0,2.0,2023-01-28,7,2.310000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-03-31,30,0.000000
2326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-03-28,109,0.000000
2327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-03-29,109,0.000000
2328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-03-30,109,0.000000


In [191]:
from sklearn.base import BaseEstimator, TransformerMixin

Classes(models) that allow us to add custom classes to the pipline
BaseEstimator - give access to get_params , and set_params 
TransformerMixin - gives us access to  .fit(), .transaform(), .fit_transform()



In [192]:
class TemporalFeaturesEngineer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):

        X_ = X.copy()

        # Generate numeric columns from datetime
        X_["hour"] = X_['pickup_hour'].dt.hour
        X_["day_of_week"] = X_['pickup_hour'].dt.dayofweek

        return X_.drop(columns=['pickup_hour'])

In [193]:
add_temporal_features = TemporalFeaturesEngineer()
add_temporal_features.fit_transform(X_train)


,rides_previous_648_hour,rides_previous_647_hour,rides_previous_646_hour,rides_previous_645_hour,rides_previous_644_hour,rides_previous_643_hour,rides_previous_642_hour,rides_previous_641_hour,rides_previous_640_hour,rides_previous_639_hour,...,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_location_id,average_rides_last_3_weeks,hour,day_of_week
0,19.0,28.0,43.0,33.0,12.0,3.0,2.0,1.0,1.0,1.0,...,4.0,5.0,3.0,8.0,18.0,22.0,4,40.920002,0,5
1,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,0.0,...,5.0,7.0,5.0,8.0,15.0,26.0,4,47.520000,0,6
2,2.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,2.0,3.0,...,1.0,5.0,2.0,1.0,2.0,0.0,4,2.970000,0,0
3,3.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,6.0,2.0,...,1.0,1.0,1.0,0.0,0.0,2.0,4,1.320000,0,1
4,3.0,16.0,28.0,21.0,12.0,5.0,4.0,2.0,2.0,3.0,...,2.0,3.0,0.0,5.0,1.0,2.0,7,2.310000,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,30,0.000000,0,4
2326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,109,0.000000,0,1
2327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,109,0.000000,0,2
2328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,109,0.000000,0,3


Creating a new class, which is inherited from baseestimateor and transformermixin classess.
Temporal feature engineering - take the data and create 2 new columns (hours, dayofweek) from the pickuphour column and the retured the dataframe
after droping the pickup hour column


In [194]:
import lightgbm as lgb

In [195]:
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    add_feature_average_rides_last_3_weeks,
    add_temporal_features,
    lgb.LGBMRegressor()
)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_3_weeks at 0x7f82860bb9d0>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [196]:
predictions = pipeline.predict(X_test)

from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=3.9389


#### HYPERPARAMETER TUNING

In [197]:
def get_pipeline(**hyperparams) -> Pipeline:      #

    # sklearn transform
    add_feature_average_rides_last_3_weeks = FunctionTransformer(
        average_rides_last_3_weeks, validate=False)

    # sklearn transform
    add_temporal_features = TemporalFeaturesEngineer()

    # sklearn pipeline
    return make_pipeline(
        add_feature_average_rides_last_3_weeks,
        add_temporal_features,
        lgb.LGBMRegressor(**hyperparams))

In [198]:
X_train_df = pd.DataFrame(X_train)
y_train_series = pd.Series(y_train)

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,     #showing the programs output, if -1 it wont show any output
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),
    }


    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train_df):

        # split data for training and validation
        X_train_, X_val_ = X_train_df.iloc[train_index, :], X_train_df.iloc[val_index,:]
        y_train_, y_val_ = y_train_series.iloc[train_index], y_train_series.iloc[val_index]

        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)

        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)

    # Return the mean score
    return np.array(scores).mean()

In [172]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5) #


[I 2023-09-15 16:35:01,812] A new study created in memory with name: no-name-b5c235b2-d0ba-4d44-b05b-bc7a10781e73


[LightGBM] [Warning] bagging_fraction is set=0.20486677082711627, subsample=1.0 will be ignored. Current value: bagging_fraction=0.20486677082711627
[LightGBM] [Warning] feature_fraction is set=0.5053337485480174, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5053337485480174
[LightGBM] [Warning] bagging_fraction is set=0.20486677082711627, subsample=1.0 will be ignored. Current value: bagging_fraction=0.20486677082711627
[LightGBM] [Warning] feature_fraction is set=0.5053337485480174, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5053337485480174


[I 2023-09-15 16:35:02,745] Trial 0 finished with value: 4.752780103890754 and parameters: {'num_leaves': 196, 'feature_fraction': 0.5053337485480174, 'bagging_fraction': 0.20486677082711627, 'min_child_samples': 95}. Best is trial 0 with value: 4.752780103890754.


[LightGBM] [Warning] bagging_fraction is set=0.7304617342123014, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7304617342123014
[LightGBM] [Warning] feature_fraction is set=0.7636848652245662, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7636848652245662
[LightGBM] [Warning] bagging_fraction is set=0.7304617342123014, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7304617342123014
[LightGBM] [Warning] feature_fraction is set=0.7636848652245662, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7636848652245662


[I 2023-09-15 16:35:04,952] Trial 1 finished with value: 3.0231948354784848 and parameters: {'num_leaves': 67, 'feature_fraction': 0.7636848652245662, 'bagging_fraction': 0.7304617342123014, 'min_child_samples': 26}. Best is trial 1 with value: 3.0231948354784848.


...

[I 2023-09-15 16:35:06,473] Trial 2 finished with value: 3.203902178821899 and parameters: {'num_leaves': 75, 'feature_fraction': 0.6949816383985841, 'bagging_fraction': 0.847562391244681, 'min_child_samples': 33}. Best is trial 1 with value: 3.0231948354784848.


[LightGBM] [Warning] bagging_fraction is set=0.30256930590574865, subsample=1.0 will be ignored. Current value: bagging_fraction=0.30256930590574865
[LightGBM] [Warning] feature_fraction is set=0.47090552628003773, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.47090552628003773
[LightGBM] [Warning] bagging_fraction is set=0.30256930590574865, subsample=1.0 will be ignored. Current value: bagging_fraction=0.30256930590574865
[LightGBM] [Warning] feature_fraction is set=0.47090552628003773, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.47090552628003773


[I 2023-09-15 16:35:07,625] Trial 3 finished with value: 2.8656888323055947 and parameters: {'num_leaves': 248, 'feature_fraction': 0.47090552628003773, 'bagging_fraction': 0.30256930590574865, 'min_child_samples': 61}. Best is trial 3 with value: 2.8656888323055947.


[LightGBM] [Warning] bagging_fraction is set=0.6544223862265155, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6544223862265155
[LightGBM] [Warning] feature_fraction is set=0.44911798381454054, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.44911798381454054
[LightGBM] [Warning] bagging_fraction is set=0.6544223862265155, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6544223862265155
[LightGBM] [Warning] feature_fraction is set=0.44911798381454054, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.44911798381454054


[I 2023-09-15 16:35:08,456] Trial 4 finished with value: 2.929740858662717 and parameters: {'num_leaves': 60, 'feature_fraction': 0.44911798381454054, 'bagging_fraction': 0.6544223862265155, 'min_child_samples': 61}. Best is trial 3 with value: 2.8656888323055947.


In [173]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 248, 'feature_fraction': 0.47090552628003773, 'bagging_fraction': 0.30256930590574865, 'min_child_samples': 61}


In [174]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] bagging_fraction is set=0.30256930590574865, subsample=1.0 will be ignored. Current value: bagging_fraction=0.30256930590574865
[LightGBM] [Warning] feature_fraction is set=0.47090552628003773, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.47090552628003773


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_3_weeks at 0x7f8282a1c310>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.30256930590574865,
                               feature_fraction=0.47090552628003773,
                               min_child_samples=61, num_leaves=248))])

In [175]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=4.2673


In [176]:
import plotly.express as px
from typing import Optional, List
from datetime import timedelta


def plot_one_sample(
    example_id: int,
    features: pd.DataFrame,
    targets: Optional[pd.Series] = None,
    predictions: Optional[pd.Series] = None
):
    """"""
    features_ = features.iloc[example_id]

    if targets is not None:
        target_ = targets.iloc[example_id]
    else:
        target_ = None

    ts_columns = [c for c in features.columns if c.startswith('rides_previous_')]
    ts_values = [features_[c] for c in ts_columns] + [target_]
    ts_dates = pd.date_range(
        features_['pickup_hour'] - timedelta(hours=len(ts_columns)),
        features_['pickup_hour'],
        freq='H'
    )

    # line plot with past values
    title = f'Pick up hour={features_["pickup_hour"]}, location_id={features_["pickup_location_id"]}'
    fig = px.line(
        x=ts_dates, y=ts_values,
        template='plotly_dark',
        title=title
    )

    if targets is not None:
        # green dot for the value we wanna predict
        fig.add_scatter(x=ts_dates[-1:], y=[target_],
                        line_color='green',
                        mode='markers', marker_size=10, name='actual value')

    if predictions is not None:
        # big red X for the predicted value, if passed
        prediction_ = predictions.iloc[example_id]
        fig.add_scatter(x=ts_dates[-1:], y=[prediction_],
                        line_color='red',
                        mode='markers', marker_symbol='x', marker_size=15,
                        name='prediction')
    return fig

In [177]:
plot_one_sample(
    example_id=40,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)